# INTRODUCTION
Providing a useful suggestion of products to online users to increase their consumption on websites is the goal of many companies nowadays. People usually select or purchase a new product based on some friend’s recommendations, comparison of similar products, or feedback from other users. In order to do all these tasks automatically, a recommender system must be implemented. The recommender systems are powerful tools that provide suggestions to the customers based on their behavior. In this project, a recommendation system using two methods will be built. The dataset that was used here consists of over 6000 movies and 3000 TV Shows. Both of them will be introduced in this project. We will select the algorithms that best fit the data and we will implement and compare them.


**Model Using Content-Based Filtering**

1. Using some features like cast, director, country, rating and genres.
2. Using the words in the movie/TV show descriptions as features

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Imports

nltk - The Natural Language Toolkit (NLTK) is a Python library for Natural Language Processing which is used for extracting words from the descrption feautres.

In [ ]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

# Loading data

We are using "Netflix Movies and TV Shows" from https://www.kaggle.com/shivamb/netflix-shows

In [ ]:
netflix_data = pd.read_csv('/content/netflix_titles.csv')
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


# Data Preprocessing

In [ ]:
#percentage of null values
netflix_data.isnull().sum()/(len(netflix_data))*100

show_id          0.000000
type             0.000000
title            0.000000
director        29.908028
cast             9.367549
country          9.435676
date_added       0.113546
release_year     0.000000
rating           0.045418
duration         0.034064
listed_in        0.000000
description      0.000000
dtype: float64

In [ ]:
#As a part preprocessing we remove the rows which have null values important features such as cast country and rating
netflix_data = netflix_data.dropna(subset=['cast', 'country', 'rating'])

# Recommendation Model 1

Using some features like *cast, director, country, rating and genres*.

Cosine similarity is the measure of similarity between two vectors, by computing the cosine of the angle between two vectors projected into multidimensional space. It can be applied to items available on a dataset to compute similarity to one another via keywords or other metrics. The similarity between two vectors (A and B) is calculated by taking the dot product of the two vectors and dividing it by the magnitude value as shown in the equation below. We can simply say that the CS score of two vectors increases as the angle between them decreases.

In [ ]:
movies = netflix_data[netflix_data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,PG-13,Comedies


In [ ]:
tv = netflix_data[netflix_data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV ..."


In [ ]:
# Extract all the actors of all the movies on Netflix
actors = []

for i in movies['cast']:
    actor = re.split(r', \s*', i) 
    actors.append(actor)
    
flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]
# if a actor in the cast of the particular movie entry in binary_actors to 1 otherwise to 0
for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1
        
binary_actors = pd.DataFrame(binary_actors).transpose()
# Follow similar procedures for Directors      
directors = []

for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()
# Follow same procedures for countries as well       
countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()
# Follow same procedures for Genre as well
genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()
# Follow same procedures for rating as well
ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [ ]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1,ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,29075,29076,29077,29078,29079,29080,29081,29082,29083,29084,29085,29086,29087,29088,29089,29090,29091,29092,29093,29094,29095,29096,29097,29098,29099,29100,29101,29102,29103,29104,29105,29106,29107,29108,29109,29110,29111,29112,29113,29114
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Follow same procedures for TV-Show as Movies
actors2 = []

for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)
    
flat_list5 = []
for sublist in actors2:
    for item in sublist:
        flat_list5.append(item)
        
actors_list2 = sorted(set(flat_list5))

binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k+=1
        
binary_actors2 = pd.DataFrame(binary_actors2).transpose()
        

countries2 = []

for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)
        
countries_list2 = sorted(set(flat_list6))

binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1
        
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1
        
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)

ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1
        
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

In [ ]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
binary2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,13531,13532,13533,13534,13535,13536,13537,13538,13539,13540,13541,13542,13543,13544,13545,13546,13547,13548,13549,13550,13551,13552,13553,13554,13555,13556,13557,13558,13559,13560,13561,13562,13563,13564,13565,13566,13567,13568,13569,13570
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Based on the input given We map with the binary_list and find the cosine similarity with each movies and print out top5
def recommender_model_1(search):
    cs_list = []
    binary_list = []
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## 4.1. Recommending Movies

In [ ]:
recommender_model_1('Kal Ho Naa Ho')

,title,director,cast,country,rating,listed_in,cos_sim
1057,Soldier,Abbas Mustan,"Rakhee Gulzar, Bobby Deol, Preity Zinta",India,TV-14,"Comedies, Dramas, International Movies",0.553399
3619,Chal Dhar Pakad,Aatmaram Dharne,Nagesh Bhonsle,India,TV-14,"Comedies, Dramas, International Movies",0.530330
4079,Irada Pakka,Kedar Shinde,"Smita Jaykar, Siddarth Jadhav",India,TV-14,"Comedies, Dramas, International Movies",0.527645
4184,Kya Kehna,Kundan Shah,"Preity Zinta, Saif Ali Khan, Anupam Kher, Fari...",India,TV-PG,"Dramas, International Movies, Romantic Movies",0.505181
718,AK vs AK,Vikramaditya Motwane,"Anil Kapoor, Anurag Kashyap",India,TV-MA,"Comedies, Dramas, International Movies",0.500000


In [ ]:
recommender_model_1("Kuch Kuch Hota Hai")

,title,director,cast,country,rating,listed_in,cos_sim
1451,Kabhi Khushi Kabhie Gham,Karan Johar,"Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Khan...",India,TV-14,"Dramas, International Movies",0.594812
1576,Hum Aapke Hain Koun,Sooraj R. Barjatya,"Madhuri Dixit, Salman Khan, Mohnish Bahl, Renu...",India,TV-14,"Classic Movies, Dramas, International Movies",0.516185
690,Hello Brother,Sohail Khan,"Salman Khan, Arbaaz Khan, Rani Mukerji, Johny ...",India,TV-14,"Action & Adventure, Comedies, International Mo...",0.500773
3619,Chal Dhar Pakad,Aatmaram Dharne,Nagesh Bhonsle,India,TV-14,"Comedies, Dramas, International Movies",0.486664
1443,Duplicate,Mahesh Bhatt,"Shah Rukh Khan, Juhi Chawla, Sonali Bendre, Fa...",India,TV-14,"Comedies, Dramas, International Movies",0.473879


In [ ]:
recommender_model_1('Rich in Love')

,title,director,cast,country,rating,listed_in,cos_sim
3002,Couples Counseling,Alexandre Reinecke,"Zezé Polessa, Daniel Dantas, Paulo Vilhena, Fe...",Brazil,TV-MA,"Comedies, Dramas, International Movies",0.470871
2855,Chadi Jawani Budhe Nu,NaN,Anand,India,TV-PG,"Comedies, International Movies, Romantic Movies",0.458349
2271,Don't Go Breaking My Heart,Johnnie To,"Louis Koo, Gao Yuanyuan, Daniel Wu",Hong Kong,TV-PG,"Comedies, International Movies, Romantic Movies",0.438763
2302,Christmas Wedding Planner,Justin G. Dyck,"Stephen Huszar, Kelly Rutherford",Canada,PG,"Comedies, International Movies, Romantic Movies",0.404226
1166,Mumbai Delhi Mumbai,Satish Rajwade,"Shiv Pandit, Piaa Bajpai",India,TV-MA,"Comedies, International Movies, Romantic Movies",0.404226


## Recommending TV shows

In [ ]:
recommender_model_1('The Blacklist')

,title,director,cast,country,rating,listed_in,cos_sim
143,The Assassination of Gianni Versace,NaN,"Edgar Ramírez, Darren Criss, Ricky Martin, Pen...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Thrillers",0.489898
673,Ozark,NaN,"Jason Bateman, Sofia Hublitz, Laura Linney, Sk...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Thrillers",0.466667
1290,Life Sentence,NaN,Lucy Hale,United States,TV-14,TV Dramas,0.461880
1721,Get Shorty,NaN,"Ray Romano, Chris O'Dowd",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.456435
1113,Unsolved,NaN,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",0.456435


In [ ]:
recommender_model_1('Friends')

,title,director,cast,country,rating,listed_in,cos_sim
1352,Episodes,NaN,"Matt LeBlanc, Tamsin Greig, Stephen Mangan, Ka...","United Kingdom, United States",TV-MA,"Classic & Cult TV, TV Comedies",0.522233
1858,Pee-wee's Playhouse,NaN,Paul Reubens,United States,TV-PG,"Classic & Cult TV, Kids' TV, TV Comedies",0.492366
1172,The Fix,NaN,"Jimmy Carr, D.L. Hughley, Katherine Ryan",United States,TV-MA,TV Comedies,0.455842
1310,The Honeymoon Stand Up Special,NaN,"Natasha Leggero, Moshe Kasher",United States,TV-MA,"Stand-Up Comedy & Talk Shows, TV Comedies",0.455842
1605,Adam Ruins Everything,NaN,"Adam Conover, Adam Lustick, Emily Axford",United States,TV-14,TV Comedies,0.455842


In [ ]:
recommender_model_1('The Vampire Diaries')

,title,director,cast,country,rating,listed_in,cos_sim
1970,The Messengers,NaN,"Shantel VanSanten, Diogo Morgado, JD Pardo, Jo...",United States,TV-14,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy",0.346688
1290,Life Sentence,NaN,Lucy Hale,United States,TV-14,TV Dramas,0.335410
1987,The Twilight Zone (Original Series),NaN,Rod Serling,United States,TV-14,"Classic & Cult TV, TV Sci-Fi & Fantasy",0.335410
41,Manifest,NaN,"Melissa Roxburgh, Josh Dallas, Athena Karkanis...",United States,TV-14,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy",0.334077
1266,The Originals,NaN,"Joseph Morgan, Daniel Gillies, Phoebe Tonkin, ...",United States,TV-14,"TV Dramas, TV Horror, TV Mysteries",0.334077


# Recommendation Model 2
Using words in the *Movie/TV show descriptions*

For paragraphs without complex punctuations and spacing, we used the built-in NLTK sentence tokenizer, called “Punkt tokenizer,” that comes with a pre-trained model. We can our own trained data models to tokenize text into sentences or therefore into words.

In [ ]:
movies_des = netflix_data[netflix_data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
movies_des.head()

,title,description
0,Sankofa,"On a photo shoot in Ghana, an American model s..."
1,The Starling,A woman adjusting to life after a loss contend...
2,Je Suis Karl,After most of her family is murdered in a terr...
3,Jeans,When the father of the man she loves insists t...
4,Grown Ups,Mourning the loss of their beloved junior high...


In [ ]:
tv_des = netflix_data[netflix_data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
tv_des.head()

,title,description
0,Blood & Water,"After crossing paths at a party, a Cape Town t..."
1,Kota Factory,In a city of coaching centers known to train I...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...
3,Dear White People,Students of color navigate the daily slights a...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...


In [ ]:
filtered_movies = []
movies_words = []

for text in movies_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    movies_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movies.append(filtered)

movies_words = [val for sublist in movies_words for val in sublist]
movies_words = sorted(set(movies_words))
movies_des['description_filtered'] = filtered_movies
movies_des.head()

,title,description,description_filtered
0,Sankofa,"On a photo shoot in Ghana, an American model s...","on photo shoot ghana , american model slips ba..."
1,The Starling,A woman adjusting to life after a loss contend...,a woman adjusting life loss contends feisty bi...
2,Je Suis Karl,After most of her family is murdered in a terr...,"after family murdered terrorist bombing , youn..."
3,Jeans,When the father of the man she loves insists t...,when father loves insists twin sons marry twin...
4,Grown Ups,Mourning the loss of their beloved junior high...,mourning loss beloved junior high basketball c...


In [ ]:
filtered_tv = []
tv_words = []

for text in tv_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    tv_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_tv.append(filtered)

tv_words = [val for sublist in tv_words for val in sublist]
tv_words = sorted(set(tv_words))
tv_des['description_filtered'] = filtered_tv
tv_des.head()

,title,description,description_filtered
0,Blood & Water,"After crossing paths at a party, a Cape Town t...","after crossing paths party , cape town teen se..."
1,Kota Factory,In a city of coaching centers known to train I...,in city coaching centers known train india ’ f...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...,a talented batch amateur bakers 10-week compet...
3,Dear White People,Students of color navigate the daily slights a...,students color navigate daily slights slippery...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...,strangers diego isabel flee home mexico preten...


In [ ]:
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k+=1
        
movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

In [ ]:
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k+=1
        
tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

In [ ]:
def recommender2(search):
    cs_list = []
    binary_list = []
    if search in movies_des['title'].values:
        idx = movies_des[movies_des['title'] == search].index.item()
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv_des['title'].values:
        idx = tv_des[tv_des['title'] == search].index.item()
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## Recommending Movies

In [ ]:

recommender2('Kal Ho Naa Ho')

,title,description,description_filtered,cos_sim
161,Team America: World Police,"In this musical satire, an all-marionette police force takes on the challenging role of keeping the peace on a troubled planet.","in musical satire , all-marionette police force takes challenging role keeping peace troubled planet .",0.410993
577,Mariposa,"New student Acha falls for Iqbal, a high-achieving student who's torn between love and family pressure.","new student acha falls iqbal , high-achieving student 's torn love family pressure .",0.410242
202,Midnight Sun,"Born with a fatal sensitivity to sunlight, a sheltered teen girl falls for her neighbor, but hides her condition from him as their romance blossoms.","born fatal sensitivity sunlight , sheltered teen girl falls neighbor , hides condition romance blossoms .",0.407223
1157,Chashme Baddoor,"When pretty new neighbor Seema falls for their shy roommate Sid, jealous womanizers Omi and Jai plot to break up the new lovebirds.","when pretty new neighbor seema falls shy roommate sid , jealous womanizers omi jai plot break new lovebirds .",0.403757
3629,Chashme Buddoor,"When pretty new neighbor Seema falls for their shy roommate Sid, jealous womanizers Omi and Jai plot to break up the new lovebirds.","when pretty new neighbor seema falls shy roommate sid , jealous womanizers omi jai plot break new lovebirds .",0.403757


In [ ]:
recommender2("Kuch Kuch Hota Hai")

,title,description,description_filtered,cos_sim
22,Naruto Shippuden: The Movie: The Lost Tower,"When Naruto is sent to recover a missing nin, the rogue manages to send him 20 years into the past, where he unites with his father to battle evil.","when naruto sent recover missing nin , rogue manages send 20 years past , unites father battle evil .",0.504647
2795,The Killer,"In lawless badlands, reclusive Cabeleira sets out to discover the fate of his gunman father and grows to be a feared assassin himself.","in lawless badlands , reclusive cabeleira sets discover fate gunman father grows feared assassin .",0.491808
3232,My Own Man,"When a man discovers he will be the father to a boy, his fear and insecurities send him on an emotional, humorous quest for his own manhood.","when discovers father boy , fear insecurities send emotional , humorous quest manhood .",0.491234
4759,Spy Time,"When his girlfriend dumps him for being a bore, a working stiff discovers that his father is a secret agent and a dangerous thug is targeting them.","when girlfriend dumps bore , working stiff discovers father secret agent dangerous thug targeting .",0.482220
1700,Baby Dolls,"When two sisters travel from Canada to Punjab to visit their maternal family, they discover an opportunity to mend a broken bond with their grandmother.","when two sisters travel canada punjab visit maternal family , discover opportunity mend broken bond grandmother .",0.478365


In [ ]:
recommender2('Rich in Love')

,title,description,description_filtered,cos_sim
5084,The Time Machine,"Based on the H.G. Wells classic, this adventure follows a scientist and inventor who's determined to prove to doubters that time travel is possible.","based h.g . wells classic , adventure follows scientist inventor 's determined prove doubters time travel possible .",0.502519
2588,Ali's Wedding,"After telling a white lie that spins out of control, the son of an Iraqi-born cleric in Melbourne becomes torn between family duty and his own heart.","after telling white lie spins control , iraqi-born cleric melbourne becomes torn family duty heart .",0.484883
4243,Love on Delivery,"To prove himself to a love interest, a delivery boy learns dubious kung fu from an aging charlatan so he can challenge his archrival to a fight.","to prove love interest , delivery boy learns dubious kung aging charlatan challenge archrival fight .",0.461880
4001,Honey 2,"Determined to prove herself as a street dancer, Maria heads from juvie to a Bronx rec center where she's introduced to her dance idol, Honey Daniels.","determined prove street dancer , maria heads juvie bronx rec center 's introduced dance idol , honey daniels .",0.431968
4618,Run,Siddharth falls in love with beautiful Jhanvi. But Jhanvi's brother Ganpat is overly possessive – and determined to destroy their relationship.,siddharth falls love beautiful jhanvi . but jhanvi 's brother ganpat overly possessive – determined destroy relationship .,0.428580


## Recommending TV shows

In [ ]:
recommender2('The Blacklist')

,title,description,description_filtered,cos_sim
1925,Stone Age,"Woody sets out to become the Stone Age's best animal trainer with a little help from his faithful pet dinosaur, Moga.","woody sets become stone age 's best animal trainer little help faithful dinosaur , moga .",0.433766
1207,Creators,"Two brilliant scientists discover the hidden virtual city of Creápolis, where citizens have the power to solve problems by believing and creating.","two brilliant scientists discover hidden virtual city creápolis , citizens power solve problems believing creating .",0.414614
1228,Monty Python Best Bits (mostly),"Prominent comedians pay tribute to Monty Python's best work in this five-part series, along with selected clips of sketches.","prominent comedians pay tribute monty python 's best work five-part series , along selected clips sketches .",0.400892
942,The King's Avatar,"When an elite gamer is forced out of his professional team, he becomes a manager of an Internet café and reinvents himself to return to online glory.","when elite gamer forced professional team , becomes manager internet café reinvents return online glory .",0.400892
1610,Alphablocks,"The letters of the alphabet come to life in Alphaland as they read, write and spell their way into an exciting world of phonic fun.","the letters alphabet life alphaland read , write spell way exciting world phonic fun .",0.400028


In [ ]:
recommender2('Friends')

,title,description,description_filtered,cos_sim
491,Thomas and Friends,"This animated children's series follows the adventures of Thomas, a cheerful blue tank engine who lives on the island of Sodor.","this animated children 's series follows adventures thomas , cheerful blue tank engine lives island sodor .",0.415241
1642,Bondi Rescue,"This reality series follows elite professional lifeguards on Sydney's Bondi Beach, as they take on everything from animal stings to criminals.","this reality series follows elite professional lifeguards sydney 's bondi beach , everything animal stings criminals .",0.403376
1301,Saint Seiya: The Lost Canvas,This anime adventure follows the battle between a saint of Athena and an avatar of Hades who's working on a painting that could destroy the world.,this anime adventure follows battle saint athena avatar hades 's working painting could destroy world .,0.401772
487,H,"At a dysfunctional hospital in Paris, three bumbling, eccentric medical employees embark on zany misadventures with surgical imprecision.","at dysfunctional hospital paris , three bumbling , eccentric medical employees embark zany misadventures surgical imprecision .",0.398790
872,BoJack Horseman,"Meet the most beloved sitcom horse of the '90s, 20 years later. He’s a curmudgeon with a heart of...not quite gold...but something like gold. Copper?","meet beloved sitcom horse '90s , 20 years later . he ’ curmudgeon heart ... quite gold ... something like gold . copper ?",0.393422


In [ ]:
recommender2('The Vampire Diaries')

,title,description,description_filtered,cos_sim
1354,Fullmetal Alchemist,"Brothers Edward and Alphonse use alchemy to resurrect their mother, but they unleash a chemical reaction that tears apart their bodies.","brothers edward alphonse use alchemy resurrect mother , unleash chemical reaction tears apart bodies .",0.439799
1355,Fullmetal Alchemist: Brotherhood,After both suffer physical damage – brothers Edward and Alphonse battle nefarious forces to try to reclaim their bodies in this supernatural anime.,after suffer physical damage – brothers edward alphonse battle nefarious forces try reclaim bodies supernatural anime .,0.436694
1177,Back Street Girls -GOKUDOLS-,"To pay for an epic blunder, three yakuza brothers are forced to alter their bodies, form a girl group and break into the underground J-Pop idol scene.","to pay epic blunder , three yakuza brothers forced alter bodies , form girl group break underground j-pop idol scene .",0.408626
1057,K.O.3an Guo,"In the Silver Dimension, Liu Bei, Guan Yu and Zhang Fei plan to enroll in an elite academy as sworn brothers, but KO One visitors derail the plan.","in silver dimension , liu bei , guan yu zhang fei plan enroll elite academy sworn brothers , ko one visitors derail plan .",0.404557
1676,Day and Night,A detective assists with an investigation into a brutal mass murder case in which his twin brother is the prime suspect.,a detective assists investigation brutal mass murder case twin brother prime suspect .,0.386783


# Conclusion

Taking the cast, director, country, rating and genres as features rather than the descriptions was definitely the better option because, most of the recommendations from descriptions were completely different genres with very little in common besides a few key words.
Hence **Recommender1 Model proved to be better.**